In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Add, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
import time

# Configuration class for model parameters
class ModelConfig:
    def __init__(self):
        self.epochs = 50
        self.batch_size = 16
        self.img_width = 224
        self.img_height = 224
        self.data_dir = "../input/train/"
        self.model_save_path = "resnet_trained_model.h5"  # Model save path

config = ModelConfig()

# Function to load image dimensions
def load_image_dimensions(directory):
    max_width = 0
    max_height = 0
    min_width = 35000
    min_height = 35000
    img_count = 0
    
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg"):
                img_count += 1
                filename = os.path.join(subdir, file)
                image = Image.open(filename)
                width, height = image.size
                if width < min_width:
                    min_width = width
                if height < min_height:
                    min_height = height
                if width > max_width:
                    max_width = width
                if height > max_height:
                    max_height = height
    
    return min_width, min_height, max_width, max_height, img_count

# Function to list directory counts
def list_directory_counts(directory):
    dir_counts = []
    for subdir, dirs, files in os.walk(directory, topdown=False):
        file_count = len(files)
        dirname = subdir
        dir_counts.append((dirname, file_count))
    return dir_counts

# Function to preprocess and split category information
def preprocess_categories(df):
    for index, row in df.iterrows():
        directory = row['Category'].split('/')
        if directory[3] != '':
            directory = directory[3]
            df.at[index, 'Category'] = directory
        else:
            df.drop(index, inplace=True)
    return df

# Function to setup data generators for training and validation
def setup_data_generators(train_dir, val_dir, img_width, img_height, batch_size):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')
    
    validation_generator = train_datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')
    
    return train_generator, validation_generator

# Function to build ResNet50 model architecture
def build_resnet50(input_shape=(224, 224, 3), num_classes=10):
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (7, 7), strides=(2, 2), name='conv1')(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f=5, filters=[32, 32, 128], stage=2, block='a', s=1)
    X = identity_block(X, 5, [32, 32, 128], stage=2, block='b')
    X = identity_block(X, 5, [32, 32, 128], stage=2, block='c')
    
    # Stage 3
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=3, block='a', s=2)
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='c')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='d')
    
    # Stage 4
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=4, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='d')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='e')
    X = identity_block(X, 3, [128, 128, 512], stage=4, block='f')
    
    # Stage 5
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=5, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=5, block='c')
    
    # Average pooling and output layer
    X = AveragePooling2D(pool_size=(5, 5), padding='valid')(X)
    X = Flatten()(X)
    X = Dense(num_classes, activation='softmax', name='fc' + str(num_classes))(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model

# Function to define convolutional block
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name_base = 'conv' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), name=conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# Function to define identity block
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'conv' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    X = Conv2D(F1, (1, 1), name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f, f), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, (1, 1), name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# Function to train the model
def train_model(model, train_generator, validation_generator, epochs, batch_size):
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Define callbacks
    checkpoint_path = "model_checkpoint.h5"
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                          save_weights_only=True,
                                          monitor='val_accuracy',
                                          mode='max',
                                          save_best_only=True)
    
    # Train the model
    history = model.fit(train_generator,
                        steps_per_epoch=train_generator.samples // batch_size,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.samples // batch_size,
                        epochs=epochs,
                        callbacks=[checkpoint_callback])
    
    # Load best weights
    model.load_weights(checkpoint_path)
    
    # Save the entire model
    model.save(config.model_save_path)
    print(f"Model saved to {config.model_save_path}")
    
    return model, history

# Function to evaluate the model
def evaluate_model(model, validation_generator):
    results = model.evaluate(validation_generator)
    return results

# Main function to run the entire pipeline
def main():
    # Load dataset dimensions
    min_width, min_height, max_width, max_height, img_count = load_image_dimensions(config.data_dir)
    print(f"Image dimensions - Min Width: {min_width}, Min Height: {min_height}, Max Width: {max_width}, Max Height: {max_height}, Total Images: {img_count}")
    
    # List directory counts
    dir_counts = list_directory_counts(config.data_dir)
    print(f"Directory Counts: {dir_counts}")
    
    # Preprocess and split category information
    df = pd.read_csv("../input/categories.csv")
    df = preprocess_categories(df)
    
    # Setup data generators for training and validation
    train_generator, validation_generator = setup_data_generators(config.data_dir, config.data_dir, config.img_width, config.img_height, config.batch_size)
    
    # Build ResNet50 model architecture
    model = build_resnet50(input_shape=(config.img_width, config.img_height, 3), num_classes=10)
    
    # Train the model
    trained_model, history = train_model(model, train_generator, validation_generator, config.epochs, config.batch_size)
    
    # Evaluate the model
    results = evaluate_model(trained_model, validation_generator)
    print(f"Evaluation Results: {results}")
    
    # Plot accuracy and loss over epochs
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

if __name__ == "__main__":
    main()
